# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv from Part I of the homework
weather_df = pd.read_csv("../Resources/weather_data_part-I.csv")
len(weather_df)
weather_dropped_df = weather_df.dropna()
len(weather_dropped_df)
print(f"Check df lenfth before and after dropping na values -\
 {len(weather_df)} vs. {len(weather_dropped_df)}") # Check if any na values
weather_df = weather_dropped_df
weather_df.head()

Check df lenfth before and after dropping na values - 558 vs. 558


,City,Temperature (DegF),Cloudiness (%),Latitude,Longitude,Humidity (%),Wind Speed (mps)
0,Rikitea,297.56,100,-23.12,-134.97,84,6.00
1,Mountain Home,289.15,90,43.13,-115.69,51,3.10
2,Dikson,275.04,100,73.51,80.55,85,4.16
3,Yambio,292.18,73,4.57,28.40,96,1.90
4,Walvis Bay,291.15,0,-22.96,14.51,17,6.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmap
gmaps.configure(api_key=g_key)

In [4]:
# Use Lat and Lng as locations and Humidity as the weight
locations = weather_df[['Latitude', 'Longitude']]
humidity  = weather_df['Humidity (%)']

In [5]:
# Add Heatmap layer to map
fig = gmaps.figure()

In [6]:
# Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=2)

In [7]:
# Add layer
fig.add_layer(heat_layer)
# fig = gmaps.figure(map_type="HYBRID")

In [8]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
fig = plt.gcf()
plt.show()
plt.draw()
fig.savefig("test.png", dpi=100)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [10]:
plt.savefig("Figures/Humidity_Heatmap.png")

<Figure size 432x288 with 0 Axes>

In [13]:
figure_layout = {
    'width': '1800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout)

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='1800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
